# Исследование надёжности заёмщиков

## 1. Обзор

Импортируем библиотеку Пандас, выводим начало таблицы и общую информацию по ней

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Наблюдения по таблице**

1. Трудовой стаж имеет дробные значения, часть из которых - отрицательные. Нужно перевести значения столбца в целочисленный тип и избавиться от отрицательных значений. 
2. Странное название столбца, где хранится возраст клиента: почему dob_years? 
3. Одни и те же значения в столбце education написаны в разном регистре. 
4. Пары столбцов education + education_id и family_status + family_status_id  - взаимозаменяемы. Похоже на то, что надо создать словари с ними. 
5. Есть пропуски в столбцах days_employed и total_income
6. Кроме столбца "days_employed" тип данных можно изменить еще и в столбце 'debt'. Логичнее использовать там булевый тип, хотя, можно оставить и так. 

**Сразу исправим то, что не будет затронуто в шаге номер 2.**

Переименуем столбец, где хранится возраст клиентов в понятный 'age'

In [3]:
data.rename(columns = {'dob_years' : 'age'}, inplace = True)

Приведем все значения в столбце 'education' к нижнему регистру и сразу выведем их, чтобы убедиться, что все сработало. 

In [4]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

## 2. Предобработка данных

### Обработка пропусков

Пропуски есть в двух столбцах, а их количество одинаково. Может ли быть такое, что пропуск в days_employed соответствует пропуску в total_income? Чтобы это проверить - посчитаем общее количество пропусков...

In [5]:
21525 - 19351

2174

...а затем посчитаем количество строк таблицы, в которых значения в столбцах 'days_employed' и 'total_income' пропущенны одновременно

In [6]:
len(data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)])

2174

2174 и 2174. Предположение подтвредилось. Допускаю, что логика простая: если ни дня не работал, то и дохода быть не могло. Возможно, при заполнении данных эти значения были взаимосвязаны. Посмотрим - вдруг значения можно восстановить, опираясь на какие-то характеристики. Выведем только эти строчки и посмотрим на них. 

In [7]:
data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)].head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Других корреляций, которые помогли бы заполнить пропуски - не вижу. Тогда действуем по дефолту.
Поскольку это пропуски в количественных переменных, то их нужно заполнить характерными значениями. Выведем среднеарифметическое и медиану с помощью describe.

In [8]:
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Интересно: максимальное значение трудового стажа - 401 тысяча дней. Это больше тысячи лет. Похоже на артефакт в данных. Попробуем разобраться и исправить. Первое, что я заметил еще при посмотре таблицы - все отрицательные значения - в пределах нормы, а вот положительные - гигантские и встречаются только у пенсионеров. Проверим это. 

In [9]:
data[data['days_employed'] > 0]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

Наблюдение подтвердилось с небольшим дополнением. Положительное количество рабочих дней в таблице - у пенсионеров и безработных. В этом прослеживается какая-то логика: и те и другие сейчас без работы. Но пока не совсем понятно. Посмотрим информацию по значениям столбца 'days_employed' для той и другой категории. 

In [10]:
data[(data['income_type'] == 'пенсионер') | (data['income_type'] == 'безработный')]['days_employed'].describe()

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64

Минимум 328 728 дней. А в каких пределах находятся отрицательные значения в этом столбце?

In [11]:
data[data['days_employed'] < 0]['days_employed'].describe()

count    15906.000000
mean     -2353.015932
std       2304.243851
min     -18388.949901
25%      -3157.480084
50%      -1630.019381
75%       -756.371964
max        -24.141633
Name: days_employed, dtype: float64

Максимальное количество рабочих дней стажа среди отрицательных значений - 18388 дней. Это 51 год. А медианное так вообще - 4.5 года. Выглядит куда правдоподобнее. Есть повод предположить, что проблема с завышенными величинами коснулась только пенсионеров и безработных. Ради интереса посмотрим - сколько лет человеку, который имеет опыт работы в 51 год.

In [12]:
data[data['days_employed'] < -18000]

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью


Начала работать с 10 лет:)

**В итоге я не смог найти ответа на то, по какой причине так вышло и почему данные записаны именно так. Какая-то логика в этом определенно есть. Вопрос скорее всего в том - как получают эти данные.** 

Для дальнейшего исследования данные именно этого столбца нам не поднадобятся, но пропуски заполнить нужно. Поскольку это количественная переменная, то заполняем пропуски харакерными значениями. Посчитаем медиану для каждой группы клиентов по источнику дохода и сохраним получившуюся серию в отдельную переменную... 

In [13]:
days_employed_median = data.groupby('income_type')['days_employed'].median()
days_employed_median

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

...а затем заменим пропуски в каждой группе получившимися значениями. Для этого - соберем все уникальные значения источников дохода, а затем пройдемся по каждому из них, заменяя пропущенные данные столбца 'days_employed' на медианы из серии 'days_employed_median'.

In [14]:
income_type_list = data['income_type'].unique()

for income_type in income_type_list:
    new_days_employed = days_employed_median[income_type]
    data[data['income_type'] == income_type] = data[data['income_type'] == income_type].fillna(value={'days_employed': new_days_employed})

Ну и заодно - уберем отрицательные значения в столбце. Отрицательный рабочий стаж в днях - это странно. 

In [15]:
data['days_employed'] = abs(data['days_employed'])

**Теперь сделаем тоже самое со столбцом total_income**

Логика та же: люди из одной группы по типу дохода скорее всего получают более близкие друг к другу суммы. Поэтому: 
- сгруппируем по типу дохода и найдем медиану для каждой группы
- сохраним в отдельную серию
- заменим все пропуски в каждой группе на значения из серии с медианами

In [17]:
total_income_median = data.groupby('income_type')['total_income'].median()

In [18]:
for income_type in income_type_list:
    new_total_income = total_income_median[income_type]
    data[data['income_type'] == income_type] = data[data['income_type'] == income_type].fillna(value={'total_income': new_total_income})

**Вывод**

Заменили пропуски в количественных данных с помощью характерных значений - медиан.
Обнаружили аномальные значения в столбце 'days_employed'.

### Замена типа данных

Тип данных нужно заменить в столбце days_employed (на целочисленный), и в столбце debt (на булевый). В нужный конкретный тип значения переводят с помощью 'as_type'.

In [20]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data['debt'] = data['debt'].astype(bool)

**Вывод**

Заменили тип данных в столбцах 'days_employed', 'total_income' и 'debt'

### Обработка дубликатов

Удалять дубликаты вручную в такой таблице - бессмысленно (и беспощадно). 
Пытаться вычислить неполные дубликаты тоже странно - любое отличие даже в одном столбце данных может означать то, что это просто другой клиент. Поэтому удалим только полные дубликаты с помощью drop_duplicates.

Для начала посмотрим - сколько у нас полных дубликатов.

In [22]:
data.duplicated().sum()

71

Посмотрим на них, чтобы попробовать выделить причины появления. 

In [23]:
data[data.duplicated() == True].head(15)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,False,142594,покупка жилья для семьи
3290,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,False,118514,сыграть свадьбу
4182,1,1574,34,высшее,0,гражданский брак,1,F,сотрудник,False,142594,свадьба
4851,0,365213,60,среднее,1,гражданский брак,1,F,пенсионер,False,118514,свадьба
5557,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,False,118514,сыграть свадьбу
6312,0,1574,30,среднее,1,женат / замужем,0,M,сотрудник,False,142594,строительство жилой недвижимости
7808,0,365213,57,среднее,1,гражданский брак,1,F,пенсионер,False,118514,на проведение свадьбы
7921,0,365213,64,высшее,0,гражданский брак,1,F,пенсионер,False,118514,на проведение свадьбы
7938,0,365213,71,среднее,1,гражданский брак,1,F,пенсионер,False,118514,на проведение свадьбы
8583,0,365213,58,высшее,0,Не женат / не замужем,4,F,пенсионер,False,118514,дополнительное образование


Интересно, что во всех показанных строках изначально не были определены значения в столбцах со стажем и доходом. На мой взгляд дубликаты могли появиться из-за того, что клиенты подавали несколько заявок на кредит и поэтому не один раз остались в базе данных. 

Удалим дубликаты (обновив индексы) и снова выведем количество дубликатов, чтобы убедиться, что все сработало. 

In [24]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**

Удалили 71 строчку с полными дубликатами. 

### Лемматизация

Леммы пригодятся нам для того, чтобы категоризировать цели получения кредита. Импортируем библиотеку для работы с лемматизацией.

In [25]:
from pymystem3 import Mystem
m = Mystem()

Создаем новый столбец и сохраняем там леммы каждой строчки. Сразу выведем таблицу и посмотрим результат работы.

In [26]:
data['purpose_lemmas'] = data['purpose'].apply(m.lemmatize)
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


Есть нюанс. Преобразуя строку в леммы, функция лемматизации сохраняет также символы пробелов и переносов на новую строку. Нам они не нужны. Напишем функцию, которая их удалит. 
Она будет получать на вход строку, считать количество пробелов в ней, а затем удалять их все, вместе с символом перехода на новую строку. 

In [27]:
def space_and_enter_remover(string):
    space_count = string.count(' ')
    for i in range(space_count):
        string.remove(' ')
    string.remove('\n')
    return string

И применим функцию ко всем значениям в столбце 'purpose_lemmas'. Снова выведем таблицу, чтобы проверить работу функции.

In [28]:
data['purpose_lemmas'] = data['purpose_lemmas'].apply(space_and_enter_remover)
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,"[покупка, жилье]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, автомобиль]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,"[покупка, жилье]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, образование]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, свадьба]"


**Вуаля! Столбец с леммами создан**

### Категоризация данных

В проекте нас ожидает работа над двумя столбцами, данные по которым нужно категоризировать. Это столбцы с целями кредитов и уровнем дохода. 

**Категоризируем данные по уровню дохода**

Посмотрим на общую информацию по данным

In [29]:
data['total_income'].describe()

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
age                 21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null bool
total_income        21454 non-null int64
purpose             21454 non-null object
purpose_lemmas      21454 non-null object
dtypes: bool(1), int64(6), object(6)
memory usage: 2.0+ MB


В интернете я нашел следующий подход к делению, который зависит от медианной зарплаты:
- меньше 0,75 медиан - низкий доход
- 0,75 - 1,25 медианы - медианная группа (средний доход)
- 1,25 - 2 медианы - выше среднего
- 2 - 4 медианы - обеспеченные
- больше 4 медиан - состоятельные

Медианная зарплата в России на 2020 год равнялась 32 422 рублей. Следовательно, наши группы будут выглядеть следующим образом (с учетом небольшого округления):

- меньше 24 т.р - низкий доход
- от 24 до 40 - средний доход
- от 40 до 65 - выше среднего
- от 65 до 130 - обеспеченные
- выше 130 - состоятельные

Попробовав разбить их таким образом я столкнулся с тем, что группы получаются очень неравномерные и состоятельных людей среди них больше всего. Поэтому используем другой способ. Разобъем данные на группы равного размера и категоризируем их. Для этого нам поднадобится метод qcut. Применим метод qcut к столбу 'total_income', разделив данные на 4 интервала и передадим ему переменную с названиями всех категорий.

In [31]:
income_categories = ['низкий', 'средний', 'выше среднего', 'обсепеченные']

data['income_category'] = pd.qcut(data['total_income'], q=4, labels=income_categories)
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,"[покупка, жилье]",обсепеченные
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, автомобиль]",средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,"[покупка, жилье]",выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, образование]",обсепеченные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, свадьба]",выше среднего


**Категоризируем данные по целям кредита**

Чтобы определить категории - посмотрим на все имеющиеся значения столбца и разделим их по смыслу. 

In [32]:
data['purpose_lemmas'].value_counts()

[автомобиль]                                  972
[свадьба]                                     791
[на, проведение, свадьба]                     768
[сыграть, свадьба]                            765
[операция, с, недвижимость]                   675
[покупка, коммерческий, недвижимость]         661
[операция, с, жилье]                          652
[покупка, жилье, для, сдача]                  651
[операция, с, коммерческий, недвижимость]     650
[покупка, жилье]                              646
[жилье]                                       646
[покупка, жилье, для, семья]                  638
[строительство, собственный, недвижимость]    635
[недвижимость]                                633
[операция, со, свой, недвижимость]            627
[строительство, жилой, недвижимость]          624
[покупка, недвижимость]                       621
[покупка, свой, жилье]                        620
[строительство, недвижимость]                 619
[ремонт, жилье]                               607


Вижу следующие уникальные группы: 
- автомобиль
- свадьба
- образование
- жилье/недвижимость(надо учитывать оба слова, ведь это одно и то же)
- коммерческая недвижимость. 

Напишем функцию, которая получает на вход строку с леммами и возвращает категорию.

In [33]:
def purpose_category_definder(purpose_lemmas):
    if ('жилье' in purpose_lemmas) or ('недвижимость' in purpose_lemmas) and ('коммерческий' not in purpose_lemmas):
        purpose_category = 'жилая недвижимость'
    elif ('коммерческий' in purpose_lemmas):
        purpose_category = 'коммерческая недвижимость'
    elif ('автомобиль' in purpose_lemmas):
        purpose_category = 'автомобиль'
    elif ('образование' in purpose_lemmas):
        purpose_category = 'образование'
    elif ('свадьба' in purpose_lemmas):
        purpose_category = 'свадьба'
    #на всякий случай оставим команду, которая присваивает иную категорию леммам, не прошедшим условия 
    #если мы увидим ее в дальнейшем, значит какие-то леммы прошли мимо условий и мы сделали ошибку
    #это будет для нас проверкой корректной работы
    else:
        purpose_category = 'иная категория'
    
    return purpose_category

Применим функцию и создадим новый столбец

In [34]:
data['purpose_category'] = data['purpose_lemmas'].apply(purpose_category_definder)

### Cловари

Пары столбцов education + education_id и family_status + family_status_id  - взаимозаменяемы. 
Похоже на то, что надо создать словари с ними. Выделим каждую пару в отдельный словарь, удалим из него дубликаты, а затем уберем строковые столбцы из общей таблицы. 

In [35]:
education_dict = data[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [36]:
family_status_dict = data[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [37]:
data.drop(['family_status', 'education'], axis=1, inplace=True)
data.head()

,children,days_employed,age,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,False,253875,покупка жилья,"[покупка, жилье]",обсепеченные,жилая недвижимость
1,1,4024,36,1,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, автомобиль]",средний,автомобиль
2,0,5623,33,1,0,M,сотрудник,False,145885,покупка жилья,"[покупка, жилье]",выше среднего,жилая недвижимость
3,3,4124,32,1,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, образование]",обсепеченные,образование
4,0,340266,53,1,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, свадьба]",выше среднего,свадьба


## 3. Анализ

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Нам нужно разбить все данные по возрастам. Для начала взглянем - какие значения вообще есть в столбце 'children'

In [38]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

А вот и еще артефакты. Значения "-1" и "20". Перед тем как считать и группировать - надо разобраться с этими данными и исправить их, отнеся к какой-то категории. Предполагаю, что значение "-1" - это на самом деле "1": либо ставили символ "тире", либо оно просто считалось в отрицательную, а не положительную сторону. Поэтому заменим все значения "-1" на "1".

In [39]:
data.loc[data['children'] < 0, 'children'] = 1

Со значением "20" все чуть-чуть сложнее. Это точно не 20 детей. Скорее всего - здесь должна быть цифра 2, но по каким-то причинам к ней прибавился ноль. Может быть это было 2.0, но что-то случилось с точкой. Так или иначе. Предлагаю перевести их в значение "2". 

In [40]:
data.loc[data['children'] == 20, 'children'] = 2
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Теперь сгруппируем всю таблицу количеству детей и сразу посчитаем два показателя: количество клиентов с задолженностью (sum) и общее количество клиентов (count). Сохраним в отдельный фрейм, чтобы работать с ним дальше и выведем

In [41]:
children_debt = data.groupby('children').agg({'debt':['sum', 'count']})
children_debt

debt       
             sum  count
children               
0         1063.0  14091
1          445.0   4855
2          202.0   2128
3           27.0    330
4            4.0     41
5            0.0      9

Уберем двойную индексацию столбцов, переименовав их, а затем найдем долю клиентов с задолженностью ('rate') среди всех по двум группам и сравним.

In [42]:
children_debt.set_axis(['has_debt', 'total'], axis='columns', inplace=True)
children_debt['rate'] = children_debt['has_debt'] / children_debt['total']

In [43]:
children_debt.sort_values(by='rate', ascending=False)

,has_debt,total,rate
children,,,
4,4.0,41,0.097561
2,202.0,2128,0.094925
1,445.0,4855,0.091658
3,27.0,330,0.081818
0,1063.0,14091,0.075438
5,0.0,9,0.000000


**Вывод**

Наличие детей влияет на возврат кредита в срок. 
Клиенты, у которых нет детей, имеют меньше фактов задолженности, нежели клиенты с детьми. Не берем в расчет клиентов с 5 детьми, потому что выборка очень маленькая.  Интересно при этом, что у клиентов с 3 детьми рейтинг задолженности получился ниже, чем у клиентов 1 или 2 детьми. 

Предполагаю, что логика здесь простая: ребенок - это дополнительная финансовая нагрузка на семью, а следовательно наличие детей (и внеплановых трат, связанных с ними) может привести с задержкам по кредиту. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Значения семейного положения мы уже проверили на этапе создания словарей. С ними все в порядке. Сгруппируем, и найдем долю клиентов с задолженностью по каждой группе. 

In [44]:
family_status_debt = data.groupby('family_status_id').agg({'debt':['sum', 'count']})
family_status_debt.set_axis(['debt', 'total'], axis='columns', inplace=True)
family_status_debt['rate'] = family_status_debt['debt'] / family_status_debt['total']
family_status_debt

,debt,total,rate
family_status_id,,,
0,931.0,12339,0.075452
1,388.0,4151,0.093471
2,63.0,959,0.065693
3,85.0,1195,0.071130
4,274.0,2810,0.097509


Смерджим получившуюся таблицу со словарем

In [45]:
family_status_debt = family_status_dict.merge(family_status_debt, on='family_status_id', how='left')

In [46]:
family_status_debt.sort_values(by='rate', ascending=False)

,family_status_id,family_status,debt,total,rate
4,4,Не женат / не замужем,274.0,2810,0.097509
1,1,гражданский брак,388.0,4151,0.093471
0,0,женат / замужем,931.0,12339,0.075452
3,3,в разводе,85.0,1195,0.071130
2,2,вдовец / вдова,63.0,959,0.065693


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок - есть. 
Чаще она встречается у клиентов без партнера или находящихся в гражданском браке. Причина может быть в том, что наличие партнера - это дополнительная финансовая поддержка, а люди в гражданском браке не так тесно связаны обязательствами друг перед другом. 
Интересно, что последнее место в нашем рейтинге заняли клиенты, потерявшие супруга. У меня есть (может быть не совсем этично прозвучит) предположение, что они получили в наследство часть имущества от своего партнера и поэтому тоже чувствуют себя более уверенно с финансовой точки зрения. А возможно - у нас просто сликшом маленькая выборка по ним. 

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Мы категоризировали клиентов по уровню дохода. Осталось лишь сгруппировать их и посчитать долю клиентов с задолженностью по кажддой группе. 

In [47]:
data.head()

,children,days_employed,age,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,False,253875,покупка жилья,"[покупка, жилье]",обсепеченные,жилая недвижимость
1,1,4024,36,1,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, автомобиль]",средний,автомобиль
2,0,5623,33,1,0,M,сотрудник,False,145885,покупка жилья,"[покупка, жилье]",выше среднего,жилая недвижимость
3,3,4124,32,1,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, образование]",обсепеченные,образование
4,0,340266,53,1,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, свадьба]",выше среднего,свадьба


In [48]:
income_debt = data.groupby('income_category').agg({'debt':['sum', 'count']})
income_debt.set_axis(['debt', 'total'], axis='columns', inplace=True)
income_debt['rate'] = income_debt['debt'] / income_debt['total']
income_debt.sort_values(by='rate', ascending=False)

,debt,total,rate
income_category,,,
средний,483.0,5479,0.088155
выше среднего,448.0,5247,0.085382
низкий,427.0,5364,0.079605
обсепеченные,383.0,5364,0.071402


**Вывод**

Интересные результаты. Клиенты с самым низким (из имеющихся) уровнем дохода имеют задолженность по кредиту реже, чем клиенты с доходом среднего и выше среднего уровня.  Звучит контринтуитивно. 
Обеспеченные клиенты, при этом, как и кажется чисто интуитивно, реже всего имеют задолженность. 

### Как разные цели кредита влияют на его возврат в срок?

Данные для этого исследования мы тоже уже категоризировали. Нам нужно сгруппировать данные и снова посчитать долю клиентов, имеющих задолженность. В этот раз сделаем это с помощью сводной таблицы, а заодно покажем долю по каждой подкатегории. 

Создадим сводную табличку, взяв туда цель кредита и ее категорию. Значениями таблицы станет факт задолженности, но обработанный функциями суммы и количества

In [50]:
purpose_debt_pivot = data.pivot_table(index=['purpose_category', 'purpose'],
                                      values='debt', 
                                      aggfunc=['sum', 'count'])

Избавимся от двойной индексации столбцов и создадим еще один, где посчитаем долю

In [51]:
purpose_debt_pivot.set_axis(['debt', 'total'], axis='columns', inplace=True)
purpose_debt_pivot['rate'] = purpose_debt_pivot['debt'] / purpose_debt_pivot['total']

А затем выведем, отсортировав по категории, а внутри каджой категории - по доле клиентов с задолженностью.

In [52]:
purpose_debt_pivot.sort_values(['purpose_category', 'rate'], ascending=[True, False])

debt  total  \
purpose_category          purpose                                               
автомобиль                сделка с автомобилем                    50.0    455   
                          сделка с подержанным автомобилем        51.0    486   
                          свой автомобиль                         48.0    478   
                          на покупку автомобиля                   44.0    471   
                          автомобили                              44.0    478   
                          приобретение автомобиля                 42.0    461   
                          на покупку своего автомобиля            46.0    505   
                          автомобиль                              42.0    494   
                          на покупку подержанного автомобиля      36.0    478   
жилая недвижимость        строительство недвижимости              54.0    619   
                          операции с недвижимостью                55.0    675   
                          покупка жилья для сдачи                 52.0    651   
                          операции со своей недвижимостью         50.0    627   
                          строительство жилой недвижимости        48.0    624   
                          покупка жилья                           48.0    646   
                          операции с жильем                       48.0    652   
                          жилье                                   46.0    646   
                          покупка жилья для семьи                 45.0    638   
                          покупка недвижимости                    43.0    621   
                          покупка жилой недвижимости              41.0    606   
                          недвижимость                            42.0    633   
                          строительство собственной недвижимости  42.0    635   
                          ремонт жилью                            35.0    607   
                          покупка своего жилья                    34.0    620   
коммерческая недвижимость операции с коммерческой недвижимостью   52.0    650   
                          покупка коммерческой недвижимости       47.0    661   
образование               получение дополнительного образования   51.0    446   
                          получение высшего образования           46.0    426   
                          профильное образование                  44.0    436   
                          заняться образованием                   39.0    408   
                          высшее образование                      40.0    452   
                          заняться высшим образованием            43.0    496   
                          получение образования                   37.0    442   
                          дополнительное образование              38.0    460   
                          образование                             32.0    447   
свадьба                   на проведение свадьбы                   64.0    768   
                          свадьба                                 64.0    791   
                          сыграть свадьбу                         58.0    765   

                                                                      rate  
purpose_category          purpose                                           
автомобиль                сделка с автомобилем                    0.109890  
                          сделка с подержанным автомобилем        0.104938  
                          свой автомобиль                         0.100418  
                          на покупку автомобиля                   0.093418  
                          автомобили                              0.092050  
                          приобретение автомобиля                 0.091106  
                          на покупку своего автомобиля            0.091089  
                          автомобиль                              0.085020  
                          на покупку подержанного автом

**Вывод**

Задолженности встречаются чаще в случае кредита на автомобиль и образование. В нижней части рейтинга - недвижимость. Допускаю, что покупка недвижимости - более осознанный шаг, к которому люди готовятся основательнее, а следовательно имеют куда более продуманный финансовый план. На "вершине" нашего рейтинга - получение дополнительного образования. Самое низкое значение у цели "покупка своего жилья". 

## 4. Выводы

Мы изучили таблицу со статистикой платежеспособности клиентов, предобработали данные в ней и проверили несколько гипотез. 

**В ходе предобработки, мы:**
- Избавлись от дубликатов
- Обработали пропуски
- Изменили тип некоторых данных
- Леммтизировали столбец целей получения кредита для дальнейшей группировки
- Создали словари с данными
- Категоризировали данные для дальнейшего анализа

**Затем провели исследование и получили следующие выводы:**

Задолженность по возврату кредита чаще встречается у:
- Клиентов с детьми
- Клиентов без партнера или находящих в гражданском браке
- Клиентов, которые берут кредит на авто и образование

Уровень дохода при этом пропорционально не влияет на уровень возврата кредита в срок. Клиенты со средним доходом имеют меньше задолженностей, нежели клиенты с доходом выше среднего. 